In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report, f1_score

import warnings
# Отключение предупреждений
warnings.filterwarnings('ignore')

/kaggle/input/cartoon/sample_submission.csv
/kaggle/input/cartoon/train.csv
/kaggle/input/cartoon/test.csv


### Классификация анимационных проектов

**Определение принадлежности видеороликов к проектам на основе анализа текстового описания, субтитров и другой доступной информации**

***Данные***

**Основное** 
- yt_reel_id уникальный идентификатор ролика
- cartoon целевая переменная

**Файлы**
- train.csv - the training set
- test.csv - the test set
- sample_submission.csv - a sample submission file in the correct format

**Колонки**
- date дата, когда ролик появился на ютубе
- reel_name название ролика
- yt_reel_id уникальный идентификатор ролика на ютубе
- cartoon название проекта, целевая переменная
- url ссылка на ролик (включает идентификатор)
- text текст сниппета, включает название ролика и описание
- seconds длительность
- is_shorts вертикальные видеоролики продолжительностью <60сек
- broadcast лайвы, прямые эфиры
- yt_channel_id идентификатор ютуб канала
- yt_channel_name название ютуб канала
- yt_ch_url ссылка на ютуб канал
- yt_channel_type тип канала (Мультфильмы, Детские, Блогеры, Shorts…)
- flag_closed ютуб канал закрыт, если 1
- international метка международных каналов, каналов на иностранном языке (переведенный - контент)
- language язык

### Загрузка данных

In [2]:
sample = pd.read_csv('/kaggle/input/cartoon/sample_submission.csv')
data = pd.read_csv('/kaggle/input/cartoon/train.csv', lineterminator='\n')
test_data = pd.read_csv('/kaggle/input/cartoon/test.csv', lineterminator='\n')

#### Посмотрим датасеты

In [3]:
def discover(data):
    display(data.info())
    display(f'Размер данных:{data.shape}') 
    display(f'Количество пропусков:{data.isna().sum()}')
    display(f'Количество дубликатов:{data.duplicated().sum()}') 
    display(data.head())

In [4]:
discover(sample) #формат рабочего решения

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55608 entries, 0 to 55607
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   yt_reel_id  55608 non-null  object
 1   cartoon     55608 non-null  object
dtypes: object(2)
memory usage: 869.0+ KB


None

'Размер данных:(55608, 2)'

'Количество пропусков:yt_reel_id    0\ncartoon       0\ndtype: int64'

'Количество дубликатов:0'

,yt_reel_id,cartoon
0,RBw4j0aMWyM,Бегемотик
1,5wTfpmlZe5k,Космохомяк
2,Fd9YA-oDjSo,Улитка
3,NXCHY_J-5v0,Космохомяк
4,XM-GRekoYzs,Космохомяк


#### Тестовая выборка

In [5]:
discover(test_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55608 entries, 0 to 55607
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             55608 non-null  object 
 1   reel_name        55605 non-null  object 
 2   yt_reel_id       55608 non-null  object 
 3   url              55608 non-null  object 
 4   text             55608 non-null  object 
 5   seconds          49807 non-null  float64
 6   is_shorts        49807 non-null  float64
 7   broadcast        49807 non-null  object 
 8   yt_channel_id    55584 non-null  object 
 9   yt_channel_name  55580 non-null  object 
 10  yt_ch_url        55580 non-null  object 
 11  yt_channel_type  55580 non-null  object 
 12  flag_closed      55580 non-null  float64
 13  international    55580 non-null  float64
 14  language         337 non-null    object 
dtypes: float64(4), object(11)
memory usage: 6.4+ MB


None

'Размер данных:(55608, 15)'

'Количество пропусков:date                   0\nreel_name              3\nyt_reel_id             0\nurl                    0\ntext                   0\nseconds             5801\nis_shorts           5801\nbroadcast           5801\nyt_channel_id         24\nyt_channel_name       28\nyt_ch_url             28\nyt_channel_type       28\nflag_closed           28\ninternational         28\nlanguage           55271\ndtype: int64'

'Количество дубликатов:0'

,date,reel_name,yt_reel_id,url,text,seconds,is_shorts,broadcast,yt_channel_id,yt_channel_name,yt_ch_url,yt_channel_type,flag_closed,international,language
0,2020-10-26,пять маленьких монстров хэллоуин рифмы потеш...,RBw4j0aMWyM,https://www.youtube.com/watch?v=RBw4j0aMWyM,[пять маленьких монстров хэллоуин рифмы поте...,960.0,0.0,none,UCfeljpDR__qqp-lLBJdiQkw,Kids Tv Russia песенки для дете,https://youtube.com/channel/UCfeljpDR__qqp-lLB...,Детские,0.0,0.0,NaN
1,2024-03-24,Tractor mini Bulldozer to making concrete road...,5wTfpmlZe5k,https://www.youtube.com/watch?v=5wTfpmlZe5k,[Tractor mini Bulldozer to making concrete roa...,0.0,0.0,live,UCIDMp2rLX4G5C6eiypl9SFA,OneDayAlex,https://www.youtube.com/channel/UCIDMp2rLX4G5C...,Блогеры,0.0,0.0,NaN
2,2020-06-28,Кто Последний ПЕРЕСТАНЕТ Ехать Выиграл Челлендж,Fd9YA-oDjSo,https://www.youtube.com/watch?v=Fd9YA-oDjSo,[Кто Последний ПЕРЕСТАНЕТ Ехать Выиграл Челле...,1599.0,0.0,none,UC2tsySbe9TNrI-xh2lximHA,A4,https://youtube.com/channel/UC2tsySbe9TNrI-xh2...,Блогеры,0.0,0.0,NaN
3,2018-07-27,Мультики про машинки и паровозики для детей. Р...,NXCHY_J-5v0,https://www.youtube.com/watch?v=NXCHY_J-5v0,[Мультики про машинки и паровозики для детей. ...,467.0,0.0,none,UC_J_9-qRfVtKIkq7SxxVbhQ,ТВ Деткам,https://youtube.com/channel/UC_J_9-qRfVtKIkq7S...,Детские,0.0,0.0,NaN
4,2021-04-03,СЛИШКОМ МАЛЕНЬКИЙ ПОРТАЛ В ЭНД В МАЙНКРАФТ ТОЛ...,XM-GRekoYzs,https://www.youtube.com/watch?v=XM-GRekoYzs,[СЛИШКОМ МАЛЕНЬКИЙ ПОРТАЛ В ЭНД В МАЙНКРАФТ ТО...,364.0,0.0,none,UCSmqZkD74x3TLt6eRM5GhTg,Dargas,https://youtube.com/channel/UCSmqZkD74x3TLt6eR...,Блогеры,0.0,0.0,NaN


#### Тренировочная выборка

In [6]:
discover(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83411 entries, 0 to 83410
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             83411 non-null  object 
 1   reel_name        83408 non-null  object 
 2   yt_reel_id       83411 non-null  object 
 3   cartoon          83411 non-null  object 
 4   url              83411 non-null  object 
 5   text             83411 non-null  object 
 6   seconds          74653 non-null  float64
 7   is_shorts        74653 non-null  float64
 8   broadcast        74653 non-null  object 
 9   yt_channel_id    83366 non-null  object 
 10  yt_channel_name  83363 non-null  object 
 11  yt_ch_url        83363 non-null  object 
 12  yt_channel_type  83363 non-null  object 
 13  flag_closed      83363 non-null  float64
 14  international    83363 non-null  float64
 15  language         497 non-null    object 
dtypes: float64(4), object(12)
memory usage: 10.2+ MB


None

'Размер данных:(83411, 16)'

'Количество пропусков:date                   0\nreel_name              3\nyt_reel_id             0\ncartoon                0\nurl                    0\ntext                   0\nseconds             8758\nis_shorts           8758\nbroadcast           8758\nyt_channel_id         45\nyt_channel_name       48\nyt_ch_url             48\nyt_channel_type       48\nflag_closed           48\ninternational         48\nlanguage           82914\ndtype: int64'

'Количество дубликатов:0'

,date,reel_name,yt_reel_id,cartoon,url,text,seconds,is_shorts,broadcast,yt_channel_id,yt_channel_name,yt_ch_url,yt_channel_type,flag_closed,international,language
0,2023-12-30,100 подарков и только один можно оставить себе,PyGZL_fCbSc,none,https://www.youtube.com/watch?v=PyGZL_fCbSc,[100 подарков и только один можно оставить себе],1854.0,0.0,none,UCcartHVtvAUzfajflyeT_Gg,Miss Katy,https://www.youtube.com/channel/UCcartHVtvAUzf...,Блогеры,0.0,0.0,NaN
1,2018-06-29,ЗАБАГАЛИ Лизу в Реальной Жизни АРТУР и НАСТЯ F...,Lcp0pFZF4eQ,none,https://www.youtube.com/watch?v=Lcp0pFZF4eQ,[ЗАБАГАЛИ Лизу в Реальной Жизни АРТУР и НАСТЯ ...,602.0,0.0,none,UCWI7puXzGTPuxsoW2HD3XxQ,Boys and Toys,https://youtube.com/channel/UCWI7puXzGTPuxsoW2...,Блогеры,0.0,0.0,NaN
2,2023-11-26,Грозная семейка Лучшие СЕСТРИНСКИЕ моменты с...,z8d6LbiBRnk,none,https://www.youtube.com/watch?v=z8d6LbiBRnk,[Грозная семейка Лучшие СЕСТРИНСКИЕ моменты ...,3604.0,0.0,none,UCw3vK8lNe5SZzL--rMgq-CQ,Nickelodeon_Cyrillic,https://youtube.com/channel/UCw3vK8lNe5SZzL--r...,Мультфильмы,0.0,0.0,NaN
3,2016-05-30,Как сделать ожерелье из ткани своими руками,0iu0p_Y8DjM,none,https://www.youtube.com/watch?v=0iu0p_Y8DjM,[Как сделать ожерелье из ткани своими руками] ...,264.0,0.0,none,UCq1JpGFxcZTbbOAz010U-og,Трум Трум,https://youtube.com/channel/UCq1JpGFxcZTbbOAz0...,Блогеры,0.0,0.0,NaN
4,2023-07-06,Sleepover Rules Om Nom Stories Cartoon for k...,rYwOVp_Edjw,none,https://www.youtube.com/watch?v=rYwOVp_Edjw,[Sleepover Rules Om Nom Stories Cartoon for ...,42899.0,0.0,none,UCniI-BQk7qAtXNmmz40LSdg,Kedoo Toons TV Funny Animations,https://youtube.com/channel/UCniI-BQk7qAtXNmmz...,Мультфильмы,0.0,0.0,NaN


In [7]:
data.isna().sum()

date                   0
reel_name              3
yt_reel_id             0
cartoon                0
url                    0
text                   0
seconds             8758
is_shorts           8758
broadcast           8758
yt_channel_id         45
yt_channel_name       48
yt_ch_url             48
yt_channel_type       48
flag_closed           48
international         48
language           82914
dtype: int64

In [8]:
data['cartoon'].value_counts() #названия мультфильмов

cartoon
none                          78231
Маша и медведь                  853
Говорящий Том                   589
Буба                            529
Кошечки собачки                 437
Енотки                          379
Бэтмен                          353
ЖилаБыла Царевна                229
Оранжевая корова                227
Зебра в клеточку                188
Фиксики                         151
Чик-Чирикино                    137
Сумка                           134
Черепашки Ниндзя                104
Чуддики                          86
My little pony                   81
Свинка Пеппа                     69
Синий трактор                    67
Чучело-Мяучело                   64
Малышарики                       60
Пороро                           51
Смешарики                        41
Приключения Пети и Волка         41
Трансформеры                     40
Простоквашино                    35
Щенячий патруль                  32
Царевны                          32
Cry babies magic tea

In [9]:
data['cartoon'].nunique() #количество проектов

45

In [10]:
data['language'].value_counts() #языки

language
english    204
spanish    201
arabic      92
Name: count, dtype: int64